## OpenAI Agent SDK


Documentation: https://openai.github.io/openai-agents-python/agents/


In [14]:
import os
from dotenv import load_dotenv
load_dotenv()

from agents import Agent, Runner
import nest_asyncio
nest_asyncio.apply()

# Access your API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
   raise ValueError("OPENAI_API_KEY not found in .env file")

True

### Basics

In [19]:
## Sync run 
agent = Agent(name="Assistant", instructions="You are a helpful assistant",model="gpt-4o-mini")
result = Runner.run_sync(agent, "Write a haiku about recursion in programming.")
print(result.final_output)

## async run 
agent = Agent(
   name="Test Agent",
   instructions="You are a helpful assistant that provides concise responses.",
   model="gpt-4o-mini"
)
result = await Runner.run(agent, "Hello! Are you working correctly?")
print(result.final_output)


Code within itself,  
A loop of endless echoes—  
Infinite unfolds.
Hello! Yes, I'm working correctly. How can I assist you today?


### Some Agent Params

### Best practices for writing instructions
- Be specific: Clearly define the agent’s role, personality, and limitations
- Set boundaries: Explicitly state what topics or actions the agent should avoid
- Define interaction patterns: Explain how the agent should handle various types of inputs
- Establish knowledge boundaries: Clarify what the agent should know and when it should acknowledge uncertainty

In [ ]:
from agents import ModelSettings

In [22]:
advanced_agent = Agent(
   name="Advanced Assistant",
   instructions="""You are a professional, concise assistant who always provides
   accurate information. When you don't know something, clearly state that.
   Focus on giving actionable insights when answering questions.""",
   model="gpt-4o-mini",
   model_settings=ModelSettings(
       temperature=0.3,  # Lower for more deterministic outputs (0.0-2.0)
       max_tokens=1024,  # Maximum length of response
   ),
   tools=[] 
)

### Agent tools use

In [23]:
from agents import WebSearchTool

In [32]:
# Create a research assistant with web search capability
research_assistant = Agent(
   name="Research Assistant",
   instructions="""You are a research assistant that helps users find and summarize information.
   When asked about a topic:
   1. Search the web for relevant, up-to-date information
   2. Synthesize the information into a clear, concise summary
   3. Structure your response with headings and bullet points when appropriate
   4. Always cite your sources at the end of your response
  
   If the information might be time-sensitive or rapidly changing, mention when the search was performed.
   """,
   tools=[WebSearchTool(
      user_location={"type": "approximate", "city": "New York"},  # Provides geographic context for local search queries
      search_context_size='medium'   ## high medium low
      )]
   )

In [33]:
# Usage example (in Jupyter notebook)
summary = Runner.run_sync(research_assistant, "Latest developments in personal productivity apps.")
print(summary)

RunResult:
- Last agent: Agent(name="Research Assistant", ...)
- Final output (str):
    As of March 25, 2025, personal productivity applications have seen significant advancements, particularly through the integration of artificial intelligence (AI) and the development of autonomous AI agents.
    
    **Integration of AI in Productivity Tools**
    
    AI has been increasingly embedded into productivity applications to enhance efficiency and user experience. Notable developments include:
    
    - **AI-Powered Assistants**: Applications like Notion have introduced AI assistants capable of drafting, summarizing, and refining content within user notes, facilitating collaborative projects and individual brainstorming. ([eti.co.ke](https://eti.co.ke/how-ai-is-changing-personal-productivity-apps-you-need/?utm_source=openai))
    
    - **Intelligent Task Management**: Tools such as Todoist have integrated AI to help users prioritize tasks based on deadlines, importance, and workload, su

#### Basic Function tool 

In [34]:
from agents import function_tool

In [35]:
@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny"

agent = Agent(
    name="Haiku agent",
    instructions="Always respond in haiku form",
    model="gpt-4o-mini",
    tools=[get_weather],
)
results = Runner.run_sync(agent, "what is the weather in new york?")
print(results.final_output)

Bright sun fills the sky,  
New York dances in warm light,  
Joy in every stride.  


#### Structured output

In [37]:
from pydantic import BaseModel
from typing import List, Optional
# Define person data model
class Person(BaseModel):
   name: str
   role: Optional[str]
   contact: Optional[str]
# Define meeting data model
class Meeting(BaseModel):
   date: str
   time: str
   location: Optional[str]
   duration: Optional[str]
# Define task data model
class Task(BaseModel):
   description: str
   assignee: Optional[str]
   deadline: Optional[str]
   priority: Optional[str]
# Define the complete email data model
class EmailData(BaseModel):
   subject: str
   sender: Person
   recipients: List[Person]
   main_points: List[str]
   meetings: List[Meeting]
   tasks: List[Task]
   next_steps: Optional[str]

In [ ]:
# Create an email extraction agent with structured output
email_extractor = Agent(
   name="Email Extractor",
   instructions="""You are an assistant that extracts structured information from emails.
  
   When given an email, carefully identify:
   - Subject and main points
   - People mentioned (names, roles, contact info)
   - Meetings (dates, times, locations)
   - Tasks or action items (with assignees and deadlines)
   - Next steps or follow-ups
  
   Extract this information as structured data. If something is unclear or not mentioned,
   leave those fields empty rather than making assumptions.
   """,
   output_type=EmailData,  # This tells the agent to return data in EmailData format
)

In [38]:
# For simple lists
agent_with_list_output = Agent(
   name="List Generator",
   instructions="Generate lists of items based on the user's request.",
   output_type=list[str],  # Returns a list of strings
)

# For dictionaries
agent_with_dict_output = Agent(
   name="Dictionary Generator",
   instructions="Create key-value pairs based on the input.",
   output_type=dict[
       str, int
   ],  # Returns a dictionary with string keys and integer values
)

# For simple primitive types
agent_with_bool_output = Agent(
   name="Decision Maker",
   instructions="Answer yes/no questions with True or False.",
   output_type=bool,  # Returns a boolean
)

- see context and output types here : https://openai.github.io/openai-agents-python/agents/

#### Agent as tool 

In [36]:
# Specialist agents
note_taking_agent = Agent(
   name="Note Manager",
   instructions="You help users take and organize notes efficiently.",
   # In a real application, this agent would have note-taking tools
)
task_management_agent = Agent(
   name="Task Manager",
   instructions="You help users manage tasks, deadlines, and priorities.",
   # In a real application, this agent would have task management tools
)
# Coordinator agent that uses specialists as tools
productivity_assistant = Agent(
   name="Productivity Assistant",
   instructions="""You are a productivity assistant that helps users organize their work and personal life.
  
   For note-taking questions or requests, use the note_taking tool.
   For task and deadline management, use the task_management tool.
  
   Help the user decide which tool is appropriate based on their request,
   and coordinate between different aspects of productivity.
   """,
   tools=[
       note_taking_agent.as_tool(
           tool_name="note_taking",
           tool_description="For taking, organizing, and retrieving notes and information"
       ),
       task_management_agent.as_tool(
           tool_name="task_management",
           tool_description="For managing tasks, setting deadlines, and tracking priorities"
       )
   ]
)
result = Runner.run_sync(productivity_assistant, "I need to keep track of my project deadlines")
print(result.final_output)

If you need assistance with implementing any of these steps, or if you'd like to discuss using specific tools, feel free to ask!
